In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
# from torch.utils.data import 
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [ ]:
# X_train_tensors = Variable(torch.Tensor(X_train))
# X_test_tensors = Variable(torch.Tensor(X_test))

# y_train_tensors = Variable(torch.Tensor(y_train))
# y_test_tensors = Variable(torch.Tensor(y_test)) 

In [ ]:


#     tensor_df = get_tensor_from_pd(df).float()
#     tensor_y = get_tensor_from_pd(y).float()
# # get_train_data()

In [ ]:
# df = torch.tensor(df.values)

In [ ]:
# y = torch.tensor(y.values)

In [ ]:

# df[1]
# df = pd.DataFrame(df)

In [ ]:
num_epochs = 1000 #1000 epochs
learning_rate = 0.001 #0.001 lr
device = torch.device("cuda")
input_size = 2 #number of features
hidden_size = 32 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
batch_size = 64
num_classes = 1 #number of output classes 

In [ ]:
# y

In [ ]:
# y[1]

In [ ]:
# df.shape

In [ ]:

df = pd.DataFrame(data=preprocessing.StandardScaler().fit_transform(np.random.randint(0, 10, size=(1000, 200))))

y = pd.Series(np.random.randint(0, 2, 500))
df = torch.tensor(df.values, dtype = float)
y = torch.tensor(y.values).reshape(500, 1)
df = df.reshape(500, 200, 2)

dataset = []

for i in range(len(df)):
    dataset.append([df[i], y[i]])
trainloader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=batch_size)

In [ ]:
df

tensor([[[ 1.1868, -0.8822],
         [ 0.5031, -0.1867],
         [-0.8365,  1.5986],
         ...,
         [-0.8896, -0.1589],
         [-0.9279,  0.8811],
         [-1.1989, -0.8618]],

        [[ 1.5367, -0.1781],
         [ 0.1602,  0.5179],
         [-1.5147, -0.1851],
         ...,
         [ 0.8554,  1.2140],
         [ 0.5051,  0.1813],
         [ 0.8910,  0.1765]],

        [[ 1.5367,  1.2299],
         [-0.5258,  1.2226],
         [-0.8365, -0.1851],
         ...,
         [ 0.1574, -1.1886],
         [ 0.5051, -1.5683],
         [ 0.1944,  1.2148]],

        ...,

        [[-1.2624,  1.2299],
         [ 1.5321, -0.8914],
         [ 1.1980, -1.6121],
         ...,
         [-1.2386, -1.1886],
         [ 1.2216, -1.5683],
         [-1.1989,  1.5609]],

        [[-0.5626,  1.2299],
         [-1.2117,  0.5179],
         [-1.1756,  1.5986],
         ...,
         [ 1.5534, -0.8454],
         [ 1.2216, -0.5186],
         [-0.8506, -1.2079]],

        [[ 0.4870, -1.2342],
       

In [ ]:
# dataset
df2 = pd.DataFrame(data=preprocessing.StandardScaler().fit_transform(np.random.randint(0, 10, size=(600, 200))))

y2 = pd.Series(np.random.randint(0, 2, 300))
df2 = torch.tensor(df2.values, dtype = float)
y2 = torch.tensor(y2.values).reshape(300, 1)
df2 = df2.reshape(300, 200, 2)
dataset2 = []
for i in range(len(df2)):
    dataset2.append([df2[i], y2[i]])
testloader = torch.utils.data.DataLoader(dataset2, shuffle=True, batch_size=batch_size)

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(Net, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) 
        self.fc = nn.Linear(128, num_classes) 

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,x):
        x = x.float()
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)) #internal state

        # Propagate input through LSTM
        
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next

        out = self.relu(hn)
        out = self.fc_1(out) 
        out = self.relu(out) 
        out = self.fc(out) 
        out = self.sigmoid(out)
        return out

In [ ]:
criterion = torch.nn.BCELoss()  # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate) 

In [ ]:
def train_epoch(model, train_loader, test_loader, num_epoch = num_epochs, criterion = criterion, optimizer = optimizer):
    model.train()
    for i in range(num_epochs):
        for seq, labels in train_loader:
            
            model = model.to(device)
            seq = seq.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            y_pred = model(seq)

            y_pred = y_pred.to(torch.float32)
            labels = labels.to(torch.float32)

            loss = criterion(y_pred, labels)
            loss.backward() 
            optimizer.step()

        if i % 100 == 0:
            # print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 
            acc = eval_performance(model, test_loader)
            print("Epoch: %d, loss: %1.5f, acc:%1.5f" % (i, loss.item(), acc)) 

In [ ]:
def eval_performance(model, test_loader):
    total_correct = 0
    total_num = 0

    model.eval()
    with torch.no_grad():
        for seq, labels in test_loader:

            seq = seq.to(device)
            labels = labels.to(device)
            
            y_pred = model(seq) 

            y_pred = y_pred > 0.5
            total_correct +=torch.eq(y_pred,labels).float().sum().item()
            total_num += seq.size(dim=0)
        acc=total_correct/total_num

    model.train()
    return acc

        
        

In [ ]:
train_epoch(lstm, trainloader, testloader)

Epoch: 0, loss: 0.69395, acc:0.55667
Epoch: 100, loss: 0.25608, acc:0.54000
Epoch: 200, loss: 0.01137, acc:0.55000
Epoch: 300, loss: 0.00137, acc:0.54667
Epoch: 400, loss: 0.00065, acc:0.54000
Epoch: 500, loss: 0.00020, acc:0.54333
Epoch: 600, loss: 0.00012, acc:0.55667
Epoch: 700, loss: 0.00005, acc:0.56000
Epoch: 800, loss: 0.00002, acc:0.55333
Epoch: 900, loss: 0.00002, acc:0.54667
